**Now, follow the process mentioned below to sign up for using the API for data collection:**

- Create a Spotify Developer account at Spotify for Developers.
- Go to Create an App and get your Client ID and Client Secret.

### Now, let’s start with the data collection task with the Spotify API. I’ll first write code to authenticate with the Spotify API and obtain an access token using the Client Credentials Flow:

In [8]:
import requests
import base64

In [10]:
# replace with your own client id and client secret
CLIENT_ID = 'c966daa5e8a746109910755539344e79'
CLIENT_SECRET = 'b441711b343142e3a953d2797b83dcf6'

# Base64 encode the client id and client secret
client_credentials = f"{CLIENT_ID}:{CLIENT_SECRET}"
client_credentials_base64 = base64.b64encode(client_credentials.encode())

# request the access token
token_url = 'https://accounts.spotify.com/api/token'
headers = {
    'Authorization': f'Basic {client_credentials_base64.decode()}'
}
data = {
    'grant_type': 'client_credentials'
}
response = requests.post(token_url, data=data, headers=headers)

if response.status_code == 200:
    access_token = response.json()['access_token']
    print("Access token obtained successfully.")
else:
    print("Error obtaining access token.")
    exit()

Access token obtained successfully.


In [11]:
!pip install spotipy

     -------------------------------------- 255.6/255.6 kB 2.0 MB/s eta 0:00:00


### Now, I’ll write a Python function to extract detailed information about each track in any Spotify playlist:

In [12]:
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyOAuth

def get_trending_playlist_data(playlist_id, access_token):
    # set up spotipy with the access token
    sp = spotipy.Spotify(auth=access_token)

    # get the tracks from the playlist
    playlist_tracks = sp.playlist_tracks(playlist_id, fields='items(track(id, name, artists, album(id, name)))')

    # extract relevant information and store in a list of dictionaries
    music_data = []
    for track_info in playlist_tracks['items']:
        track = track_info['track']
        track_name = track['name']
        artists = ', '.join([artist['name'] for artist in track['artists']])
        album_name = track['album']['name']
        album_id = track['album']['id']
        track_id = track['id']

        # get audio features for the track
        audio_features = sp.audio_features(track_id)[0] if track_id != 'Not available' else None

        # get release date of the album
        try:
            album_info = sp.album(album_id) if album_id != 'Not available' else None
            release_date = album_info['release_date'] if album_info else None
        except:
            release_date = None

        # get popularity of the track
        try:
            track_info = sp.track(track_id) if track_id != 'Not available' else None
            popularity = track_info['popularity'] if track_info else None
        except:
            popularity = None

        # add additional track information to the track data
        track_data = {
            'Track Name': track_name,
            'Artists': artists,
            'Album Name': album_name,
            'Album ID': album_id,
            'Track ID': track_id,
            'Popularity': popularity,
            'Release Date': release_date,
            'Duration (ms)': audio_features['duration_ms'] if audio_features else None,
            'Explicit': track_info.get('explicit', None),
            'External URLs': track_info.get('external_urls', {}).get('spotify', None),
            'Danceability': audio_features['danceability'] if audio_features else None,
            'Energy': audio_features['energy'] if audio_features else None,
            'Key': audio_features['key'] if audio_features else None,
            'Loudness': audio_features['loudness'] if audio_features else None,
            'Mode': audio_features['mode'] if audio_features else None,
            'Speechiness': audio_features['speechiness'] if audio_features else None,
            'Acousticness': audio_features['acousticness'] if audio_features else None,
            'Instrumentalness': audio_features['instrumentalness'] if audio_features else None,
            'Liveness': audio_features['liveness'] if audio_features else None,
            'Valence': audio_features['valence'] if audio_features else None,
            'Tempo': audio_features['tempo'] if audio_features else None,
            # add more attributes as needed (go through the documentation to know what more you can add)
        }

        music_data.append(track_data)

    # create a pandas dataframe from the list of dictionaries
    df = pd.DataFrame(music_data)

    return df

### Now, let’s use our function get_trending_playlist_data using a specific Spotify playlist ID and an already obtained access token:

In [13]:
# you can add the playlist id of any playlist on Spotify here
playlist_id = '1gfWsOG1WAoxNeUMMktZbq'

# call the function to get the music data from the playlist and store it in a DataFrame
music_df = get_trending_playlist_data(playlist_id, access_token)

print(music_df)

                                     Track Name  \
0                                 Bijlee Bijlee   
1                                   Expert Jatt   
2   Kaun Nachdi (From "Sonu Ke Titu Ki Sweety")   
3                                   Na Na Na Na   
4                                   Patiala Peg   
..                                          ...   
95                                                
96                               Move Your Lakk   
97                    Patola (From "Blackmail")   
98            Ban Ja Rani (From "Tumhari Sulu")   
99          Hauli Hauli (From "De De Pyaar De")   

                                    Artists  \
0                             Harrdy Sandhu   
1                                     Nawab   
2                Guru Randhawa, Neeti Mohan   
3                                    J Star   
4                            Diljit Dosanjh   
..                                      ...   
95                                            
96  Diljit 

In [14]:
music_df.to_csv("musicdata.csv")